In [3]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [4]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\CATASTRO_S13.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [5]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

In [6]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [7]:
gdf_cat = gpd.read_file(PATH_CAT)
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,obs,procesado
0,48,"MULTIPOINT (502997.553 8098186.723, 503193.105...",41594,AGROPECUARIA CAMPO DULCE S.R.L.,None,False
1,49,"MULTIPOINT (517642.135 8100562.919, 517729.895...",41594,AGROPECUARIA CAMPO DULCE S.R.L.,None,False


# CARGAR PARTE DIARIO

In [8]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[48, 49]

In [9]:
gdf_result = gdf_intersect_result[['geometry', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)

In [10]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
1908,"POLYGON ((503002.471 8097559.099, 503337.155 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L13.1,6.382570,48,-1,50
2686,"POLYGON ((517473.104 8099697.511, 517470.020 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-5,46.927453,49,6,50
2687,"POLYGON ((517284.872 8100427.273, 517305.369 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-7,40.032043,49,6,50
2689,"POLYGON ((517325.558 8100269.710, 518372.975 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-6,41.673738,49,6,50
3352,"POLYGON ((517957.736 8100828.278, 517959.485 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-8,15.717325,49,6,50
5870,"POLYGON ((503458.817 8097146.051, 503304.124 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L11,21.207110,48,6,50
5871,"POLYGON ((503343.969 8096821.402, 503476.683 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L12.3,4.656979,48,6,50
7111,"POLYGON ((517631.019 8099070.346, 517616.004 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-3,48.050826,49,6,50
7117,"POLYGON ((518143.797 8098838.509, 518930.078 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-2,57.214863,49,4,50
7118,"POLYGON ((519067.992 8098208.248, 518672.874 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-1,45.320456,49,4,50


In [11]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [12]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
1908,"POLYGON ((503002.471 8097559.099, 503337.155 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L13.1,6.382570,48,-1,50
2686,"POLYGON ((517473.104 8099697.511, 517470.020 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-5,46.927453,49,6,50
2687,"POLYGON ((517284.872 8100427.273, 517305.369 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-7,40.032043,49,6,50
2689,"POLYGON ((517325.558 8100269.710, 518372.975 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-6,41.673738,49,6,50
3352,"POLYGON ((517957.736 8100828.278, 517959.485 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-8,15.717325,49,6,50
5870,"POLYGON ((503458.817 8097146.051, 503304.124 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L11,21.207110,48,6,50
5871,"POLYGON ((503343.969 8096821.402, 503476.683 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L12.3,4.656979,48,6,50
7111,"POLYGON ((517631.019 8099070.346, 517616.004 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-3,48.050826,49,6,50
7117,"POLYGON ((518143.797 8098838.509, 518930.078 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-2,57.214863,49,4,50
7118,"POLYGON ((519067.992 8098208.248, 518672.874 8...",480,EL CANAL,41594,AGROPECUARIA CAMPO DULCE S.R.L.,Ca-1,45.320456,49,4,50


In [13]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [14]:
os_para_cargar

[48, 49]

In [15]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 48 marcado como procesado.
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 49 marcado como procesado.
